In [58]:
from os import urandom
from hashlib import sha512
from Crypto.Cipher import AES

import copy
import os

In [59]:
def DeriveKeyIV(password, salt, key_length, iv_length):
    KeyAndIV_0 = b''  # changed '' to b''
    KeyAndIV_1 = copy.copy(KeyAndIV_0)  
    while len(KeyAndIV_0) < key_length + iv_length:
        KeyAndIV_1 = sha512(KeyAndIV_1 + str.encode(password) + salt).digest()
        KeyAndIV_0 += KeyAndIV_1
    return KeyAndIV_0[:key_length], KeyAndIV_0[key_length:key_length+iv_length]

In [60]:
def encrypt(input_file_path, output_file_path_0, output_file_path_1, piece, password, salt_no, salt_header='', key_length=32):
    with open(input_file_path, 'rb') as in_file, open(output_file_path_0, 'wb') as out_file_1, open(output_file_path_1, 'wb') as out_file_2:
        bs = AES.block_size  # best not to change
        iv_length = AES.block_size  # best not to change
        salt = urandom(salt_no - len(salt_header))
        key, iv = DeriveKeyIV(password, salt, key_length, iv_length)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        out_file_1.write(str.encode(salt_header) + salt)
        finished = False
        n=0  # index for separate the output files
        while not finished:
            chunk = in_file.read(piece * bs) 
            if len(chunk) == 0 or len(chunk) % bs != 0:
                padding_length = (bs - len(chunk) % bs) or bs
                chunk += str.encode(padding_length * chr(padding_length))
                finished = True
            if n%2 == 0:
                out_file_1.write(cipher.encrypt(chunk))
            if n%2 == 1:
                out_file_2.write(cipher.encrypt(chunk))
            n += 1
def decrypt(input_file_path_0, input_file_path_1, output_file_path, piece, password, salt_no, salt_header='', key_length=32):
    with open(input_file_path_0, 'rb') as in_file_0, open(input_file_path_1, 'rb') as in_file_1, open(output_file_path, 'wb') as out_file:
        bs = AES.block_size  # best not to change
        iv_length = AES.block_size  # best not to change
        salt = in_file_0.read(salt_no)[len(salt_header):]
        key, iv = DeriveKeyIV(password, salt, key_length, iv_length)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        next_chunk = ''
        finished = False
        n=0  # index for separate the output files
        while not finished:
            if n%2 == 0:
                chunk, next_chunk = next_chunk, cipher.decrypt(in_file_0.read(piece * bs))
            if n%2 == 1:
                chunk, next_chunk = next_chunk, cipher.decrypt(in_file_1.read(piece * bs))

            if len(next_chunk) == 0:
                padding_length = chunk[-1]  # removed ord(...) as unnecessary
                chunk = chunk[:-padding_length]
                finished = True 

            out_file.write(bytes(x for x in chunk))  # changed chunk to bytes(...)
            n += 1

## Demo

In [61]:
# parameters
piece = 10
password = 'default'
salt_no = 100
salt_header = 'default'

### encrypt

In [62]:
# path
input_file_path = 'test.txt'
output_file_path_0 = '1.txt'
output_file_path_1 = '2.txt'

In [63]:
# main
encrypt(input_file_path, output_file_path_0, output_file_path_1, piece, password, salt_no, salt_header)

### decrypt

In [64]:
# path
input_file_path_0 = '1.txt'
input_file_path_1 = '2.txt'
output_file_path = 'test_combine.txt'

In [65]:
# main
decrypt(input_file_path_0, input_file_path_1, output_file_path, piece, password, salt_no, salt_header)